In [10]:
import os
import plotly.express as px
import pandas as pd

In [11]:
import os
import pandas as pd
import plotly.express as px

def count_images_in_folders(root_folder):
    records = []
    
    for subset in ['train', 'test', 'val']:
        subset_path = os.path.join(root_folder, subset)
        
        for category in os.listdir(subset_path):
            category_path = os.path.join(subset_path, category)
            
            if os.path.isdir(category_path):
                num_images = len([f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))])
                
                records.append({'Category': category, 'Count': num_images, 'Dataset': subset})

    return pd.DataFrame(records)

root_folder = "car_make_images"
df = count_images_in_folders(root_folder)

# Sort and plot
df = df.sort_values(['Dataset', 'Count'], ascending=[True, True])
fig = px.bar(df, x='Count', y='Category', color='Dataset', orientation='h', title='Distribution of Car Makes', 
             height=1000, width=800)
fig.show()


In [12]:
import pandas as pd
hist = pd.read_csv('history.csv')
hist.columns = ['Epoch', 'Loss', 'Accuracy', 'f1_score', 'Val_Loss', 'Val_Accuracy', 'Val_f1_score']

In [13]:
hist

,Epoch,Loss,Accuracy,f1_score,Val_Loss,Val_Accuracy,Val_f1_score
0,1,3.722818,0.040353,0.000259,3.630959,0.046214,0.000000
1,2,3.622794,0.042081,0.000000,3.614736,0.049413,0.000000
2,3,3.617013,0.045364,0.000000,3.662155,0.047991,0.000000
3,4,3.616401,0.045364,0.000000,3.602281,0.050480,0.000000
4,5,3.607370,0.048734,0.000000,3.592560,0.047991,0.000000
...,...,...,...,...,...,...,...
235,236,1.998353,0.433077,0.394419,3.486670,0.277284,0.202631
236,237,2.007543,0.437570,0.397625,3.419580,0.283327,0.199076
237,238,2.005826,0.435410,0.400663,3.362030,0.272663,0.197654
238,239,2.002101,0.432299,0.397184,3.387492,0.281905,0.206897


In [14]:
import plotly.express as px
acc = px.line(hist[['Accuracy', 'Val_Accuracy']])

In [15]:
import plotly.express as px
f1 = px.line(hist[['f1_score', 'Val_f1_score']])

In [16]:
import plotly.express as px
loss = px.line(hist[['Loss', 'Val_Loss']])

In [17]:
import time
import plotly.express as px
import pandas as pd
import re

def create_plotly_figure(dataframe, columns, fig_name):
    print("Creating figure:", fig_name)
    start_time = time.time()
    fig = px.line(dataframe[columns])
    print("Created figure, now writing...")
    fig.write_html(f"../assets/figures/{fig_name}.html")
    print(f"Figure {fig_name} created and written. Time taken: {time.time() - start_time}")

def update_readme():
    print("Starting to update README...")
    start_time = time.time()

    # Read existing README.md
    with open("../README.md", "r", encoding='utf-8') as f:
        content = f.read()

    # Load data
    print("Loading data...")
    hist = pd.read_csv('history.csv')
    hist.columns = ['Epoch', 'Loss', 'Accuracy', 'f1_score', 'Val_Loss', 'Val_Accuracy', 'Val_f1_score']
    
    # Create and save new Plotly figures
    figures_data = [
        {'data': hist, 'columns': ['Loss', 'Val_Loss'], 'name': 'loss'},
        {'data': hist, 'columns': ['Accuracy', 'Val_Accuracy'], 'name': 'acc'},
        {'data': hist, 'columns': ['f1_score', 'Val_f1_score'], 'name': 'f1'}
    ]
    
    figures_md = "\nMétricas actuales del modelo:\n"
    for figure in figures_data:
        create_plotly_figure(figure['data'], figure['columns'], figure['name'])
        figures_md += f"### {figure['name'].capitalize()} Plot\n"
        figures_md += f"![{figure['name'].capitalize()} Plot](../assets/figures/{figure['name']}.html)\n"

    # Use regex to replace the existing Metrics section with the new figures
    print("Updating README...")
    updated_content = re.sub(r"Métricas actuales del modelo:.*", figures_md, content, flags=re.DOTALL)

    # Write the modified content back to README.md
    with open("../README.md", "w", encoding='utf-8') as f:
        f.write(updated_content)

    print(f"README updated. Total time taken: {time.time() - start_time}")

update_readme()

Starting to update README...
Loading data...
Creating figure: loss
Created figure, now writing...
Figure loss created and written. Time taken: 0.04871392250061035
Creating figure: acc
Created figure, now writing...
Figure acc created and written. Time taken: 0.054964542388916016
Creating figure: f1
Created figure, now writing...
Figure f1 created and written. Time taken: 0.04850625991821289
Updating README...
README updated. Total time taken: 0.1608433723449707
